In [ ]:
library(twang)
library(weights)
library(tidyverse)

In [ ]:
data_dir <- file.path("..", "data")

In [ ]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds")) %>%
    mutate(sunday = factor(as.integer(icu_adm_weekday == 0), levels = c(0, 1)),
           monday = factor(as.integer(icu_adm_weekday == 1), levels = c(0, 1)),
           tuesday = factor(as.integer(icu_adm_weekday == 2), levels = c(0, 1)),
           wednesday = factor(as.integer(icu_adm_weekday == 3), levels = c(0, 1)),
           thursday = factor(as.integer(icu_adm_weekday == 4), levels = c(0, 1)),
           friday = factor(as.integer(icu_adm_weekday == 5), levels = c(0, 1)),
           saturday = factor(as.integer(icu_adm_weekday == 6), levels = c(0, 1))) %>%
    mutate(female = factor(as.integer(as.integer(as.character(gender)) == 0), levels = c(0, 1)))
head(full_data)

In [ ]:
library(tableone)

In [ ]:
tab_features <- CreateTableOne(vars = c("vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
                                        "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
                                        "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
                                        "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
                                        "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
                                        "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               strata = "echo",
                               factorVars = c("lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               data = full_data)
tab_features$ContTable

In [ ]:
tab_basics <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                                      "saps", "sofa", "vent", "vaso", "sedative",
                                      "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                      "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                      "icu_adm_weekday"),
                             strata = "echo",
                             factorVars = c("gender", "vent", "vaso", "sedative",
                                            "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                            "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                            "icu_adm_weekday"),
                             data = full_data) %>%
    print

In [ ]:
full_data %>% { split(select(., age, weight), .$echo) } %>%
setNames(c("fst", "sec")) %>%
                {list(x = { pull(pluck(., "fst"), age) },
                         y = { pull(pluck(., "sec"), age) },
                         weight = { pull(pluck(., "fst"), weight) },
                         weighty = { pull(pluck(., "sec"), weight) }, samedata = FALSE)} %>%
do.call(wtd.t.test, .) %>% .$coefficients %>%
                .["p.value"] %>%
                data.frame(wtd_p_val = .)

In [ ]:
c("age", "female", "weight", "elix_score",
  "saps", "sofa", "vent", "vaso", "sedative",
  "icd_chf", "icd_afib", "icd_renal", "icd_liver",
  "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
  "sunday", "monday", "tuesday", "wednesday",
  "thursday", "friday", "saturday",
  "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
  "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
  "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
  "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
  "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
  "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag") %>%
map(function(var) {
    x <- full_data[[var]]
    sym_var <- rlang::sym(var)
    if (is.numeric(x)) {
        res <- full_data %>%
            group_by(echo) %>%
            summarise(main = mean(!!sym_var, na.rm = TRUE),
                      sub = sd(!!sym_var, na.rm = TRUE),
                      wtd_main = wtd.mean(!!sym_var, weight),
                      wtd_sub = sqrt(wtd.var(!!sym_var, weight))) %>%
            cbind(
                full_data %>%
                { split(.[[var]], .$echo) } %>%
                setNames(c("x", "y")) %>%
                do.call(t.test, .) %>%
                .$p.value %>%
                data.frame(p_val = .)
            ) %>%
            cbind(
                full_data %>%
                { split(select(., !!sym_var, weight), .$echo) } %>%
                setNames(c("fst", "sec")) %>%
                {
                    list(x = { pull(pluck(., "fst"), !!sym_var) },
                         y = { pull(pluck(., "sec"), !!sym_var) },
                         weight = { pull(pluck(., "fst"), weight) },
                         weighty = { pull(pluck(., "sec"), weight) },
                         samedata = FALSE) 
                }
                %>%
                do.call(wtd.t.test, .) %>%
                .$coefficients %>%
                .["p.value"] %>%
                unname %>%
                data.frame(wtd_p_val = .)
            )
    }
    
    if (is.factor(x)) {
#         return(NULL)
        if (length(levels(x)) == 2) {
            res <- full_data %>%
                mutate(!!sym_var := as.integer(as.character(!!sym_var))) %>%
                group_by(echo) %>%
                summarise(main = mean(!!sym_var, na.rm = TRUE) * 100,
                          sub = NA,
                          wtd_main = wtd.mean(!!sym_var, weight) * 100,
                          wtd_sub = NA) %>%
                cbind(
                    full_data %>%
                    {
                        list(x = pull(., !!sym_var),
                             y = pull(., echo))
                    } %>%
                    do.call(chisq.test, .) %>%
                    pluck("p.value") %>%
                    unname %>%
                    data.frame(p_val = .)
                ) %>%
                cbind(
                    full_data %>%
                    {
                        list(var1 = pull(., !!sym_var),
                             var2 = pull(., echo),
                             weight = pull(., weight))
                    } %>%
                    do.call(wtd.chi.sq, .) %>%
                    pluck("p.value") %>%
                    unname %>%
                    data.frame(wtd_p_val = .)
                )
        } else return(NULL)
    }
    
    res %>%
    gather("key", "value", main, sub, wtd_main, wtd_sub) %>%
    unite("key", key, echo) %>%
    spread(key, value) %>%
    mutate(var = var) %>%
    return
}) %>%
discard(is.null) %>%
data.table::rbindlist() %>%
as.data.frame